In [1]:
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf
from model import PnlpMixerSeqCls, PnlpMixerTokenCls
from mixer import FFFTrainFixed
from dataset import PnlpMixerDataModule
from run import PnlpMixerSeqClsTrainModule, PnlpMixerTokenClsTrainModule
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import time
import torch.nn as nn

/mnt/raid/miniconda3/envs/to_cpp/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from FFF import FFFInference
class ThreeLayerFFFT(nn.Module):
    def __init__(self, input_size, hidden_size, num_blocks):
        super(ThreeLayerFFFT, self).__init__()
        self.layer1 = FFFTrainFixed(input_size, hidden_size, num_blocks)
        self.layer2 = FFFTrainFixed(input_size, hidden_size, num_blocks)
        self.layer3 = FFFTrainFixed(input_size, hidden_size, num_blocks)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
three_layer_ffft = ThreeLayerFFFT(2048, 2048, 4).to("cpu")

in_t = torch.rand(2048).to("cpu")

three_layer_ffft.eval()

# single_layer = FFFInference(three_layer_ffft.layer2)
# three_layer_ffft.layer2 = single_layer
# single_layer = FFFInference(three_layer_ffft.layer1)
# three_layer_ffft.layer1 = single_layer
# single_layer = FFFInference(three_layer_ffft.layer3)
# three_layer_ffft.layer3 = single_layer

# with torch.no_grad():
#     start = time.time()
#     res = three_layer_ffft(in_t)
#     end = time.time()
# print("původní ffft znovu", end-start)

ThreeLayerFFFT(
  (layer1): FFFTrainFixed(
    (linear_in): Linear(in_features=2048, out_features=31, bias=False)
    (linear_out): Linear(in_features=31, out_features=2048, bias=False)
    (activation): GELU()
  )
  (layer2): FFFTrainFixed(
    (linear_in): Linear(in_features=2048, out_features=31, bias=False)
    (linear_out): Linear(in_features=31, out_features=2048, bias=False)
    (activation): GELU()
  )
  (layer3): FFFTrainFixed(
    (linear_in): Linear(in_features=2048, out_features=31, bias=False)
    (linear_out): Linear(in_features=31, out_features=2048, bias=False)
    (activation): GELU()
  )
)

In [3]:
%%timeit
with torch.no_grad():
    res = three_layer_ffft(in_t)

In [ ]:
single_layer = FFFInference(three_layer_ffft.layer2)
three_layer_ffft.layer2 = single_layer
single_layer = FFFInference(three_layer_ffft.layer1)
three_layer_ffft.layer1 = single_layer
single_layer = FFFInference(three_layer_ffft.layer3)
three_layer_ffft.layer3 = single_layer

In [ ]:
%%timeit
with torch.no_grad():
    res = three_layer_ffft(in_t)

33.7 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
